In [2]:
from loaders.Loaders import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from featureSelection.baseFeatureSelector import LowVarianceFS
from splitters.splitters import SingletaskStratifiedSplitter
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import roc_auc_score, precision_score, accuracy_score, confusion_matrix, classification_report
from parameterOptimization.HyperparameterOpt import HyperparamOpt_Valid
from imbalanced_learn.ImbalancedLearn import SMOTEENN

from sklearn.ensemble import RandomForestClassifier

In [3]:
#Load Dataset
dataset = CSVLoader(dataset_path='preprocessed_dataset_wfoodb.csv', 
                    mols_field='Smiles', 
                    labels_fields='Class', 
                    id_field='ID')#, shard_size=4000)
dataset = dataset.create_dataset()
print(dataset.get_shape())

Mols_shape:  23290
Features_shape:  X not defined!
Labels_shape:  (23290,)
None


In [4]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Featurizing datapoint 5000
Featurizing datapoint 6000


RDKit ERROR: [13:33:17] Explicit valence for atom # 1 Cl, 4, is greater than permitted


error in smile: O=[Cl]=O
Featurizing datapoint 7000


RDKit ERROR: [13:33:21] Explicit valence for atom # 3 B, 4, is greater than permitted
RDKit ERROR: [13:33:21] Explicit valence for atom # 1 Cl, 9, is greater than permitted


error in smile: OB1O[B]2(O)OB(O)O[B](O)(O1)O2
error in smile: O=[Cl-](=O)(=O)=O
Featurizing datapoint 8000
Featurizing datapoint 9000
Featurizing datapoint 10000
Featurizing datapoint 11000
Featurizing datapoint 12000
Featurizing datapoint 13000
Featurizing datapoint 14000


RDKit ERROR: [13:33:36] Explicit valence for atom # 0 P, 11, is greater than permitted


error in smile: [P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C
Featurizing datapoint 15000
Featurizing datapoint 16000
Featurizing datapoint 17000
Featurizing datapoint 18000
Featurizing datapoint 19000
Featurizing datapoint 20000
Featurizing datapoint 21000
Featurizing datapoint 22000
Featurizing datapoint 23000
Elements with indexes:  [6257, 7708, 7709, 14244]  were removed due to the presence of NAs!
The elements in question are:  ['O=[Cl]=O' 'OB1O[B]2(O)OB(O)O[B](O)(O1)O2' 'O=[Cl-](=O)(=O)=O'
 '[P](OCC=C(C)C)(OCC=C(C)C)(=O)(OP(OCC=C(C)C)(OCC=C(C)C)=O)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)(CC=C(C)C)CC=C(C)C']
Mols_shape:  23286
Features_shape:  (23286, 1024)
Labels_shape:  (23286,)


In [5]:
#Feature Selection
dataset = LowVarianceFS(0.15).featureSelection(dataset)
dataset.get_shape()

Mols_shape:  23286
Features_shape:  (23286, 49)
Labels_shape:  (23286,)


In [6]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

In [7]:
train_dataset = SMOTEENN().sample(train_dataset)
train_dataset.get_shape()

Mols_shape:  13968
Features_shape:  (22795, 49)
Labels_shape:  (22795,)


In [8]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators, max_features, class_weight):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, 
                                      class_weight=class_weight)
    return rf_model

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "class_weight": [{0: 1., 1: 1.}, {0: 1., 1: 5}, {0: 1., 1: 10}]
                  }
    

In [9]:
#Hyperparameter Optimization
optimizer = HyperparamOpt_Valid(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, train_dataset, 
                                                                     valid_dataset, Metric(roc_auc_score))

print('#################')
print(best_hyperparams)
print(best_rf)

MODE:  classification
Fitting 15 random models from a space of 24 possible models.
Fitting model 1/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 1.0}}
expected str, bytes or os.PathLike object, not NoneType
roc_auc_score: 
 0.7294304194626952
Model 1/15, Metric roc_auc_score, Validation set 1: 0.729430
	best_validation_score so far: 0.729430
Fitting model 2/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 5}}
expected str, bytes or os.PathLike object, not NoneType
roc_auc_score: 
 0.7392775751952424
Model 2/15, Metric roc_auc_score, Validation set 2: 0.739278
	best_validation_score so far: 0.739278
Fitting model 3/15
hyperparameters: {'n_estimators': 10, 'max_features': 'auto', 'class_weight': {0: 1.0, 1: 10}}
expected str, bytes or os.PathLike object, not NoneType
roc_auc_score: 
 0.7338017880156731
Model 3/15, Metric roc_auc_score, Validation set 3: 0.733802
	best_validation_score so far: 0.739

In [10]:
#Evaluate model
metrics = [Metric(roc_auc_score), Metric(precision_score), Metric(accuracy_score), Metric(confusion_matrix), 
           Metric(classification_report)]
best_rf.evaluate(test_dataset, metrics)

roc_auc_score: 
 0.7333802740100697
precision_score: 
 0.32710280373831774
accuracy_score: 
 0.9117268041237113
confusion_matrix: 
 [[4105  288]
 [ 123  140]]
classification_report: 
               precision    recall  f1-score   support

           0       0.97      0.93      0.95      4393
           1       0.33      0.53      0.41       263

    accuracy                           0.91      4656
   macro avg       0.65      0.73      0.68      4656
weighted avg       0.93      0.91      0.92      4656



{'roc_auc_score': 0.7333802740100697,
 'precision_score': 0.32710280373831774,
 'accuracy_score': 0.9117268041237113,
 'confusion_matrix': 1164.0,
 'classification_report': None}